In [ ]:
# Install libraries
%pip install pandas 
%pip install top2vec
%pip install numpy

%pip install matplotlib
%pip install umap-learn[plot]
%pip install top2vec[sentence_encoders]


In [1]:
from read_corpus_functions import load_books_blocks_from_document, load_books_from_document, load_books_chunks_from_document, get_pos, discard_words
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from top2vec import Top2Vec
import umap.plot
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import pickle

/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/uma

#### Folders

In [2]:
path = "data/lem/all"
file_name =   "jerome_synv9_all_blocks_without_residuals_new" #"jerome_synv9_id_no_names_blocks"  "jerome_synv9_id" #_NOV_MEM_COL_deep_learn
file_path = path+"/"+file_name + ".txt"
fiction = True
txtype_select = ['NOV: próza',  'MEM: memoáry, autobiografie', 'COL: kratší próza', 'VER: poezie'] if fiction else [] #,
folder = 'whole books  NOV COL MEM VER' if fiction else 'whole books'
file_ending = "_NOV_COL_MEM_VER" if fiction else ''
jerome_conllu =  "data/jerome corpus/jerome_synv9_id.txt"

#books = trim_documents(books, 10000)

In [3]:
#with open('ner_combined.obj', 'rb') as f:
#        names_frame = pickle.load(f)
#        df_ner = pd.DataFrame(names_frame)

#with open('pos_NOV_COL_MEM_VER.obj', 'rb') as f:
#        work_pos = pickle.load(f)    

books, books_info = load_books_from_document(file_path, txtype_select)
#books = discard_words(['český', 'československý', 'čestina', 'česko', 'česky'], books)

In [ ]:
#len(books)
pos = get_pos( jerome_conllu, txtype_select, work_pos, df_ner)   
pickle.dump(pos, open('pos_NOV_COL_MEM_VER.obj', 'wb'))    


### CODES FOR VISUALIZATION

In [24]:
def heat_map_visualization_3(df):
    languages = df.srclang.unique()
    topics = np.unique(df.topic.unique())   
    heat_mat = np.zeros(shape=(len(topics), 3))
    for i, topic in enumerate(topics):
        for j, language in enumerate(languages):
            # print(str(topic) + ' ' + str(language) )
            # print(len(df[(df.topic == topic) & (df.srclang == language)].index)   )
            if language in ['cs: čeština']: #, 'en: angličtina'
                heat_mat[i][0] = len(df[(df.topic == topic) & (df.srclang == language)].index) 
            elif language in ['en: angličtina']: #, 
                heat_mat[i][1] = len(df[(df.topic == topic) & (df.srclang == language)].index)   
            else:
                heat_mat[i][2] += len(df[(df.topic == topic) & (df.srclang == language)].index)     
        heat_mat[i][0] = heat_mat[i][0] /  len(df[(df.topic == topic)])  
        heat_mat[i][1] = heat_mat[i][1] /  len(df[(df.topic == topic)])
        heat_mat[i][2] = heat_mat[i][2] /  len(df[(df.topic == topic)])
    return heat_mat


def heat_map_visualization_2(df):
    languages = df.srclang.unique()
    topics = np.unique(df.topic.unique()) 
    heat_mat = np.zeros(shape=(len(topics), 2))
    for i, topic in enumerate(topics):
        for j, language in enumerate(languages):
            if language in ['cs: čeština']: 
                heat_mat[i][0] = len(df[(df.topic == topic) & (df.srclang == language)])   
            else:
                heat_mat[i][1] += len(df[(df.topic == topic) & (df.srclang == language)])    
        heat_mat[i][1] = heat_mat[i][1] /  len(df[(df.topic == topic)])
        heat_mat[i][0] = heat_mat[i][0] /  len(df[(df.topic == topic)])  
    return heat_mat    


def plot_heat_map(heat_mat, df, save_name, r, labels):
    plt.figure(figsize=(15, 12))
    fig = plt.imshow(heat_mat, cmap='magma_r', interpolation='nearest')
    plt.xticks(range(r), labels)
    num_topics = len(df.topic.unique()) 
    y_ticks = ['Topic ' + str(n) for n in range(num_topics)]
    plt.yticks(range(num_topics), y_ticks)
    plt.colorbar(fig)
    plt.savefig("plots/whole books/top2vec/heat_map_{}_{}".format(save_name, str(r)))


def plot_top2vec(n_neighbors, topic_model, save_name, doc_top_reduced):

    umap_args_model = {
    "n_neighbors": n_neighbors,
    "n_components": 2,
    "metric": "cosine",
    'min_dist':0.1,
    'spread':1,
    'random_state': 42
    }

    umap_model = umap.UMAP(**umap_args_model).fit(topic_model.document_vectors)
    umap.plot.points(umap_model, labels=doc_top_reduced ) #
    plt.title(save_name)
    plt.savefig("plots/whole books/top2vec/plot_{}".format(save_name)) 



### CREATE MODELS/ SAVE MODELS AND VISUALIZATION

In [4]:
for embedding_name in ["doc2vec"]: #, 'doc2vec','paraphrase-multilingual-MiniLM-L12-v2', 'distiluse-base-multilingual-cased'
    for u_map_n_neighbours in range(15,16, 5):
        umap_args = {'n_neighbors':u_map_n_neighbours, 
                      'n_components':5, 
                      'metric':'cosine', 
                      'random_state':42}
        for hdbscan_min_cluster_size in range(15, 16,  5):
            
            hdbscan_args = {'min_cluster_size':hdbscan_min_cluster_size, 
                            'metric':'euclidean', 
                            'cluster_selection_method':'eom', 
                            'prediction_data':True}
            
            topic_model = Top2Vec(documents = books, umap_args=umap_args, hdbscan_args=hdbscan_args, speed='fast-learn',embedding_model= embedding_name, workers = 8) #
            save_name = 'without_cs_top2vec_umap_n_n{}_hdbscan_m_c_s{}_{}_new{}'.format(str(u_map_n_neighbours), str(hdbscan_min_cluster_size), str(embedding_name), file_ending)
            #save_name = 'top2vec_original_setting_new_NOV_COL_VER'
            #topic_model = Top2Vec.load('models/whole books/top2vec/{}'.format(save_name))
            topic_model.save('models/{}/top2vec/{}'.format(folder, save_name))
            num_topics = topic_model.get_num_topics()

            topic_words, word_scores, topics = topic_model.get_topics(num_topics=num_topics) # Specify the number of topics 
            
            df_topic_words = pd.DataFrame({"topic {i}".format(i=i):topic_words[i, :] for i in range(num_topics)}) #{"topic 1":topic_words[0, :], "topic 2":topic_words[1, :]}
            df_topic_words.to_excel("data/topic words/top2vec/{}/{}.xlsx".format(folder, save_name))     
            
            
            topic_sizes, topic_nums = topic_model.get_topic_sizes()
            doc_top_reduced = [0] * (len(books_info))
            for topic_num in range(num_topics): 
                print(topic_num)
                documents, document_scores, document_ids = topic_model.search_documents_by_topic(topic_num=topic_num, num_docs=topic_sizes[topic_num])
                for document_id, document_score in zip(document_ids, document_scores):
                    print(document_id)
                    doc_top_reduced[document_id] = topic_num 
                    books_info[document_id]['topic'] = topic_num 
                    books_info[document_id]['score'] = document_score

            df = pd.DataFrame(books_info)        

            df.to_excel('data/results/top2vec/{}/{}.xlsx'.format(folder, save_name))

            heat_mat_3 = heat_map_visualization_3(df)
            plot_heat_map(heat_mat_3, df, save_name, 3, ['cz', 'en', 'other'])

            heat_mat_2 = heat_map_visualization_2(df)
            plot_heat_map(heat_mat_2, df, save_name, 2, ['cz', 'trans'])
            #plot_top2vec(u_map_n_neighbours, topic_model, save_name, doc_top_reduced)
            

2024-06-25 15:24:29,787 - top2vec - INFO - Pre-processing documents for training
2024-06-25 15:24:56,141 - top2vec - INFO - Creating joint document/word embedding
2024-06-25 15:38:38,376 - top2vec - INFO - Creating lower dimension embedding of documents
2024-06-25 15:38:42,214 - top2vec - INFO - Finding dense areas of documents
2024-06-25 15:38:42,230 - top2vec - INFO - Finding topics


0
194
28
278
24
158
357
108
280
25
293
170
421
39
256
255
126
315
6
323
286
213
48
22
27
311
16
429
21
74
219
73
596
395
60
353
182
415
82
58
71
26
93
75
88
304
289
9
89
454
317
308
55
51
23
407
63
231
171
209
202
279
434
426
340
334
98
223
162
106
365
379
296
405
349
300
208
155
236
492
229
115
337
284
674
342
302
151
212
46
222
78
177
354
355
348
261
285
389
399
329
433
146
200
527
559
159
276
569
17
268
598
547
92
270
316
655
274
143
335
191
104
99
164
85
142
257
273
347
116
20
368
47
312
19
320
294
138
134
301
70
50
580
103
0
238
461
438
579
1
597
234
382
376
259
341
291
157
419
331
281
481
682
533
199
84
439
168
271
263
437
563
180
248
410
137
305
386
34
411
401
77
687
297
552
107
518
113
145
120
453
128
272
686
361
443
322
169
251
307
79
110
408
587
130
369
125
117
356
102
392
396
101
4
471
153
403
548
463
184
530
283
583
613
277
601
572
183
397
133
574
582
457
204
639
477
589
681
269
599
338
43
621
602
299
136
536
398
603
18
591
373
590
38
64
467
1
446
319
139
189
627
505
543
61

NameError: name 'heat_map_visualization_3' is not defined

### SAVE TOPIC WORDS

In [3]:


def get_TF_IDF(documents):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(documents)
    #for ele1, ele2 in zip(vectorizer.get_feature_names_out(), vectorizer.idf_):
    #    print(ele1, ':', ele2)
    return X.to_array()


 


In [ ]:
for embedding_name in ["doc2vec"]: #, 'doc2vec','paraphrase-multilingual-MiniLM-L12-v2', 'distiluse-base-multilingual-cased'
    for u_map_n_neighbours in range(15,16, 5):
        umap_args = {'n_neighbors':u_map_n_neighbours, 
                      'n_components':5, 
                      'metric':'cosine', 
                      'random_state':42}
        for hdbscan_min_cluster_size in range(15, 16,  5):
            
            hdbscan_args = {'min_cluster_size':hdbscan_min_cluster_size, 
                            'metric':'euclidean', 
                            'cluster_selection_method':'eom', 
                            'prediction_data':True}
            
            save_name = 'top2vec_umap_n_n{}_hdbscan_m_c_s{}_{}_new'.format(str(u_map_n_neighbours), str(hdbscan_min_cluster_size), str(embedding_name)) #_NOV_COL_MEM_VER
            #save_name = 'top2vec_original_setting_new_NOV_COL_MEM_VER'
            #topic_model = Top2Vec.load('models/whole books/top2vec/{}'.format(save_name))
            topic_model = Top2Vec.load('models/whole books/top2vec/{}'.format(save_name)) #  NOV COL MEM VER
            num_topics = topic_model.get_num_topics()

            topic_words, word_scores, topics = topic_model.get_topics(num_topics=num_topics) # Specify the number of topics you want

            df_topic_words = pd.DataFrame({"topic {i}".format(i=i):topic_words[i, :] for i in range(num_topics)}) #{"topic 1":topic_words[0, :], "topic 2":topic_words[1, :]}
            df_topic_words.to_excel("data/topic words/top2vec/whole books/{}.xlsx".format(save_name))   #  NOV COL MEM VER   
            
            
            topic_sizes, topic_nums = topic_model.get_topic_sizes()
            doc_top_reduced = [0] * (len(books_info))
            for topic_num in range(num_topics): 
                print(topic_num)
                documents, document_scores, document_ids = topic_model.search_documents_by_topic(topic_num=topic_num, num_docs=topic_sizes[topic_num])
                tf_idf_vector = get_TF_IDF(documents)
                print(tf_idf_vector)
                for document_id, document_score in zip(document_ids, document_scores):
                    print(document_id)
                    doc_top_reduced[document_id] = topic_num 
                    books_info[document_id]['topic'] = topic_num 
                    books_info[document_id]['score'] = document_score
                    

            df = pd.DataFrame(books_info)        

            #df.to_excel('data/results/top2vec/whole books  NOV COL MEM VER/{}.xlsx'.format(save_name))

            #heat_mat_3 = heat_map_visualization_3(df)
            #plot_heat_map(heat_mat_3, df, save_name, 3, ['cz', 'en', 'other'])

            #heat_mat_2 = heat_map_visualization_2(df)
            #plot_heat_map(heat_mat_2, df, save_name, 2, ['cz', 'trans'])



### GET X for RANDOM FOREST

In [5]:
from stop_words import get_stop_words

stop_words = get_stop_words('czech')
#stop_words.extend(['aby', 'být'])

X = []
y = []

X_structural = {'ids': [], 'tfidf' : [], 'pos': [], 'avg_sentence_length': [], 'number_of_sentences':[], 'topic': [] }

documents_list = []
for embedding_name in ["doc2vec"]: #, 'doc2vec','paraphrase-multilingual-MiniLM-L12-v2', 'distiluse-base-multilingual-cased'
    for u_map_n_neighbours in range(15,16, 5):
        umap_args = {'n_neighbors':u_map_n_neighbours, 
                      'n_components':5, 
                      'metric':'cosine', 
                      'random_state':42}
        
        for hdbscan_min_cluster_size in range(15, 16,  5):
            
            tf_idf_vectors = []
            hdbscan_args = {'min_cluster_size':hdbscan_min_cluster_size, 
                            'metric':'euclidean', 
                            'cluster_selection_method':'eom', 
                            'prediction_data':True}
            
            save_name = 'without_cs_top2vec_umap_n_n{}_hdbscan_m_c_s{}_{}_new{}'.format(str(u_map_n_neighbours), str(hdbscan_min_cluster_size), str(embedding_name), file_ending)
            #save_name = 'top2vec_original_setting_new_NOV_COL_VER'
            #topic_model = Top2Vec.load('models/whole books/top2vec/{}'.format(save_name))
            topic_model = Top2Vec.load('models/{}/top2vec/{}'.format(folder, save_name)) #  NOV COL MEM VER
            num_topics = topic_model.get_num_topics()
            
            topic_sizes, topic_nums = topic_model.get_topic_sizes()
            for topic_num in range(num_topics): 
                print(topic_num)
                documents, document_scores, document_ids = topic_model.search_documents_by_topic(topic_num=topic_num, num_docs=topic_sizes[topic_num])
                documents = [' '.join([x for x in document.split()]) for document in documents.tolist()] #if x not in stop_words else ''  <-- get rid of stop words
                documents_list.append(' '.join(documents))
                X.extend(documents)
                y.extend([topic_num for x in range(len(documents))])
                #print(tf_idf_vector)

               
 

0
1
2
3


In [ ]:
filtered_tf_idf = []
for i, l in enumerate(tf_idf_vectors):
    all_but_l = sum(tf_idf_vectors[:i] + tf_idf_vectors[i+1:], [])
    filtered = list(set(l).difference(set(all_but_l)))
    filtered_tf_idf.append(filtered) 
    print(filtered)

### ÜMIT RANDOMFOREST CODE

In [6]:
n_gram = 1

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,n_gram) )),
    ('rf', RandomForestClassifier())
])

# how can i see what the tokenizer is doing?
tokenized = pipeline.named_steps['tfidf'].fit_transform(X)
# show tokenized as text
tokenized_text = pipeline.named_steps['tfidf'].inverse_transform(tokenized)
print(tokenized_text[:110])
pipeline.fit(X, y)

# Get feature names from TfidfVectorizer
feature_names = pipeline.named_steps['tfidf'].get_feature_names_out()

# Get feature importances by class
importances_by_class = {}
for class_index, class_label in enumerate(pipeline.named_steps['rf'].classes_):
    print('Class {}'.format(class_label))
    # Binary classification: one class vs rest
    y_binary = (y == class_label)
    rf_binary = RandomForestClassifier(oob_score=classification_report)
    rf_binary.fit(pipeline.named_steps['tfidf'].transform(X), y_binary)
    importances_by_class[class_label] = dict(
        sorted(zip(feature_names, rf_binary.feature_importances_), key=lambda x: x[1], reverse=True))

# Print feature importances by class
for class_label, importances in importances_by_class.items():
    print(f"\nClass: {class_label}")
    for feature_name, importance in list(importances.items())[:25]:
        print(f"{feature_name}: {importance}")

# report classification score on the oob set
print(f"oob score: {rf_binary.oob_score_}")   

[array(['změknout', 'ucucnout', 'čistota', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['rozhodování', 'zodpovídat', 'učení', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['totožný', 'urputnost', 'soutěživost', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['stísněně', 'psaný', 'urovnat', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['uvolněně', 'atlas', 'kapesní', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['obezřetně', 'přirůst', 'znávat', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['malina', 'baže', 'péci', ..., 'jít', 'být', 'pak'], dtype='<U50'), array(['uveřejnění', 'dlaha', 'znehybněný', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['odmlčal', 'breptat', 'pff', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['vyvolání', 'vycpávat', 'zbavený', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['pompidou', 'miniatura', 'účelový', ..., 'jít', 'být', 'pak'],
      dtype='<U50'), array(['tetovaný', 'skóre',

/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

### Save results

In [7]:
df_dict_rf = {}
for class_label, importances in importances_by_class.items():
    print(f"\nClass: {class_label}")
    df_dict_rf[class_label] = list(importances.keys())[:50]

df_random_forrest_topics = pd.DataFrame(df_dict_rf) 
df_random_forrest_topics.to_excel("data/topic words/top2vec/{}/random_forrest_n_gram_{}_with_sw_{}.xlsx".format(folder, n_gram, save_name))



Class: 0

Class: 1

Class: 2

Class: 3


### BERTopic code for topic clusters

In [13]:
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired

ctfidf_model = ClassTfidfTransformer(bm25_weighting=False, reduce_frequent_words=True)

vectorizer_model = CountVectorizer(ngram_range=(1,2)) 

print('Vectorizer model')
X2 = vectorizer_model.fit_transform(documents_list)

print('Get words')
words = vectorizer_model.get_feature_names_out()

print('ctfidf_model')
ctfidf_model = ctfidf_model.fit(X2, multiplier=None)

print('transform')
c_tf_idf = ctfidf_model.transform(X2)

Vectorizer model
Get words
ctfidf_model
transform


### From BERTopic code

In [14]:
def _top_n_idx_sparse(matrix, n: int) -> np.ndarray:
        """ Return indices of top n values in each row of a sparse matrix

        Retrieved from:
            https://stackoverflow.com/questions/49207275/finding-the-top-n-values-in-a-row-of-a-scipy-sparse-matrix

        Arguments:
            matrix: The sparse matrix from which to get the top n indices per row
            n: The number of highest values to extract from each row

        Returns:
            indices: The top n indices per row
        """
        indices = []
        for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
            n_row_pick = min(n, ri - le)
            values = matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]]
            values = [values[index] if len(values) >= index + 1 else None for index in range(n)]
            indices.append(values)
        return np.array(indices)

def _top_n_values_sparse(matrix, indices: np.ndarray) -> np.ndarray:
        """ Return the top n values for each row in a sparse matrix

        Arguments:
            matrix: The sparse matrix from which to get the top n indices per row
            indices: The top n indices per row

        Returns:
            top_values: The top n scores per row
        """
        top_values = []
        for row, values in enumerate(indices):
            scores = np.array([matrix[row, value] if value is not None else 0 for value in values])
            top_values.append(scores)
        return np.array(top_values)

In [17]:
top_n_words = 50
indices = _top_n_idx_sparse(c_tf_idf, top_n_words)
scores = _top_n_values_sparse(c_tf_idf, indices)
sorted_indices = np.argsort(scores, 1)
indices = np.take_along_axis(indices, sorted_indices, axis=1)
scores = np.take_along_axis(scores, sorted_indices, axis=1)

# topics = {label: [(words[word_index], score)
#                           if  score > 0 #word_index is not None and
#                           else ("", 0.00001)
#                           for word_index, score in zip(indices[label][::-1], scores[label][::-1])
#                           ]
#                   for label in range(num_topics)}
# topics = {label: values[:top_n_words] for label, values in topics.items()}





representation_model = KeyBERTInspired()
topics = representation_model.extract_topics(documents_list, c_tf_idf, topics)

TypeError: KeyBERTInspired.extract_topics() missing 1 required positional argument: 'topics'

#### KeyBERT Inspired 

In [18]:

labels = [int(label) for label in sorted(list(topics.keys()))]


topics = {
            label: [
                (words[word_index], score) if word_index is not None and score > 0 else ("", 0.00001)
                for word_index, score in zip(indices[index][::-1], scores[index][::-1])
            ]
            for index, label in enumerate(labels)
        }
topics = {label: list(zip(*values[: 50]))[0] for label, values in topics.items()}

In [19]:
topics[2]


('se',
 'být',
 'ten',
 'on',
 'na',
 'že',
 'do',
 'ale',
 'já',
 'mít',
 'který',
 'co',
 'oni',
 'když',
 'tak',
 'jako',
 'svůj',
 'všechen',
 'být se',
 'za',
 'moci',
 'už',
 'aby být',
 'aby',
 'ty',
 'jen',
 'jak',
 'říci',
 'být ten',
 'po',
 'jeho',
 'my',
 'ten být',
 'ještě',
 'chtít',
 'muset',
 'vědět',
 'jít',
 'jeden',
 'pán',
 'ani',
 'že být',
 'se on',
 'vidět',
 'můj',
 'pak',
 'až',
 'stát',
 'než',
 'člověk')

In [11]:
df_bertopics = pd.DataFrame(topics) 
df_bertopics.to_excel("data/topic words/top2vec/{}/bertopic_{}.xlsx".format(folder, save_name))

In [12]:
num_topics = topic_model.get_num_topics()

topic_words, word_scores, topics = topic_model.get_topics(num_topics=num_topics) # Specify the number of topics you want

df_topic_words = pd.DataFrame({"topic {i}".format(i=i):topic_words[i, :] for i in range(num_topics)})

In [10]:
print(c_tf_idf[4].shape)
len(words)
label = 3
top_n_words = 50
indices = _top_n_idx_sparse(c_tf_idf, top_n_words)

for en, score in enumerate(c_tf_idf[label][::-1]):
    print('{}, {}'.format(en, score[0][0]))


(1, 7485327)
0,   (0, 7485261)	0.008974776719429589
  (0, 7485258)	0.008023700338373662
  (0, 7485226)	0.008974776719429589
  (0, 7485225)	0.008974776719429589
  (0, 7485210)	0.008974776719429589
  (0, 7485199)	0.008974776719429589
  (0, 7485180)	0.00867291096751175
  (0, 7485167)	0.008974776719429589
  (0, 7485166)	0.008974776719429589
  (0, 7485165)	0.008974776719429589
  (0, 7485164)	0.008974776719429589
  (0, 7485161)	0.008974776719429589
  (0, 7485160)	0.008974776719429589
  (0, 7485157)	0.008974776719429589
  (0, 7485148)	0.008974776719429589
  (0, 7485134)	0.008974776719429589
  (0, 7485133)	0.008974776719429589
  (0, 7485132)	0.008474077079817684
  (0, 7485131)	0.008974776719429589
  (0, 7485111)	0.008974776719429589
  (0, 7485110)	0.008974776719429589
  (0, 7485109)	0.00867291096751175
  (0, 7485100)	0.008974776719429589
  (0, 7485095)	0.008974776719429589
  (0, 7485091)	0.007459053289752938
  :	:
  (0, 320)	0.008974776719429589
  (0, 319)	0.008974776719429589
  (0, 300)	0.008

In [22]:
from collections import Counter 

u_map_n_neighbours = 15
hdbscan_min_cluster_size = 15
embedding_name = 'doc2vec'
save_name = 'top2vec_umap_n_n{}_hdbscan_m_c_s{}_{}_new{}'.format(str(u_map_n_neighbours), str(hdbscan_min_cluster_size), str(embedding_name), file_ending)
df_topic_results = pd.read_excel('data/results/top2vec/{}/{}.xlsx'.format(folder,save_name))
df_info = pd.read_excel('data/books info/stats/jerome_all_books_info.xlsx')
if fiction: df_info = df_info[df_info['txtype'].isin(txtype_select)].reset_index()
df_info['topic'] = df_topic_results['topic']
df_info['score'] = df_topic_results['score']
pos_percentage = [{key: value/len(pos_tags) for key, value in Counter(pos_tags).items()} for pos_tags in pos.values()]# [{key: value/len(pos_tags)} for pos_tags in pos.values() for key, value in Counter(pos_tags).most_common() ]
#df_info['pos'] = 
#df_info['pos'] = df_info['pos'].apply(lambda x: (key, value/len(x)) for key, value in x )


In [23]:
print(len(pos_percentage))

717


In [49]:
pos_dict = {'N': 'substantivum',
            'A': 'adjektivum', 
            'P': 'pronomen',
            'C': 'numerál', 
            'V': 'verbum',
            'D': 'adverbium',
            'R': 'prepozice',
            'J': 'konjunkce',
            'T': 'partikule',
            'I': 'interjekce',
            'S': 'segment',
            'B': 'zkratka',
            'Z': 'interpunkce', 
            'F': 'cizí slovo',
            'X': 'neznámý'}

def replace_string(string):
    for key, value in pos_dict.items():
        string = string.replace(key, value)
    return string

In [48]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize


# Texts divided into topic categories 
topic_texts = { 'id': [], 'documents': [], 'tokens':[], 'number_of_sentences': [], 'words': [], 'average_length_of_sentence': [], 'average_pos':[], 'topic': [], 'tfidf': [], 'pos_tfidf': [] }


vectorizer = TfidfVectorizer()
X_tfidf_text = vectorizer.fit_transform(books).toarray()

vectorizer = TfidfVectorizer()
pos_list = [' '.join(value) for value in pos.values()]
pos_list = [replace_string(value) for value in pos_list]

X_tfidf_pos = vectorizer.fit_transform(pos_list).toarray()

for i, row in df_info.iterrows():
    document = books[i]
    topic_texts['documents'].append(document)   
    topic_texts['words'].append(row.words)
    topic_texts['tokens'].append(row.tokens)
    topic_texts['topic'].append(row.topic)
    topic_texts['average_pos'].append(pos_percentage[i])
    topic_texts['tfidf'].append(X_tfidf_text[i])
    topic_texts['pos_tfidf'].append(X_tfidf_pos[i])
    topic_texts['id'].append(i)
    sentences = sent_tokenize(document)
    words = word_tokenize(document)
    topic_texts['number_of_sentences'] = len(sentences)
    topic_texts['average_length_of_sentence'] = len(words) / len(sentences) 

topic_texts = pd.DataFrame(topic_texts)    

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charlottepanuskova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(topic_texts[['pos_tfidf']], topic_texts['topic']) #'pos_tfidf',  'tfidf'

RandomForestClassifier(random_state=42)

In [54]:
clf.feature_importances_

array([1., 0., 0.])

In [32]:
topic_texts[["tokens", "number_of_sentences", "average_length_of_sentence", "average_pos", "tfidf"]]

TypeError: unhashable type: 'list'